In [1]:
import numpy as np
import sys
import psycopg2
from tftb.generators import amgauss, fmlin
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig
from tftb.processing import WignerVilleDistribution

In [2]:
cmd = r'"C:\Users\Ian Rain-Water\Jupyter\He6CRESDAQ\He6DAQ-develop"'
sys.path.append(cmd)

In [3]:
from Control_Logic import Data_Quality_Control as DQC
from Control_Logic import PostgreSQL_Interface as he6db

In [22]:
specCMD = 'Freq_data_2022-05-13-01-05-42.spec'
specArray = DQC.spec_to_array(specCMD, 4096, 1000)
smallSpecArray = specArray[3090]
#print(np.shape(smallSpecArray))
#print(specArray[4095])

In [5]:
plt.figure()
plt.plot(smallSpecArray)
plt.show();

In [23]:
f_stft, t_stft, Zxx = sig.stft(smallSpecArray)
Zxx = np.fft.fftshift(Zxx, axes = 0)
f_stft = np.fft.fftshift(f_stft)
wvd = WignerVilleDistribution(smallSpecArray)
tfr_wvd, t_wvd, f_wvd = wvd.run()
#f_wvd is the normalized frequencies
print(np.shape(Zxx))

(129, 9)


In [24]:
fig, axx = plt.subplots(2,1)
df1 = f_stft[1] - f_stft[0] #freq step
im = axx[0].imshow(np.real(Zxx[1:2] * np.conj(Zxx)), aspect = 'auto',
                  interpolation = None, origin = 'lower')
axx[0].set_ylabel('frequency [Hz]')
plt.colorbar(im, ax=axx[0])
axx[0].set_title('spectrogram')
#plots the spectrogram in theory, but runs into issues because the stft is 3D not sure if I can ignore any dimension in it

f_wvd = np.fft.fftshift(np.fft.fftfreq(tfr_wvd.shape[0]))
df_wvd = f_wvd[1]-f_wvd[0] #frequency change in the WVT
im = axx[1].imshow(np.fft.fftshift(tfr_wvd, axes = 0), aspect = 'auto', origin = 'lower')
axx[1].set_xlabel('time [s]')
axx[1].set_ylabel('frequency [Hz]')
plt.colorbar(im, ax=axx[1])
axx[1].set_title('Wigner-Ville Transform')
plt.show()